## ¿Qué problema se va a resolver?
El tema de investigación surgió por la necesidad que tienen las empresas operadoras de taxis de la ciudad de Bogotá, estas vienen presentando una creciente problemática de indisponibilidad de servicios de taxis en sus plataformas, esto suceso ocurre principalmente en algunas zonas u horarios específicos.
La empresa Taxis Libres (TL), una de las más grandes operadoras de servicios de taxis de la ciudad, ha empezado a recibir una gran cantidad de inconformidades por partes de los usuarios recurrentes de la plataforma por esta misma causa. Solo en el mes de mayo del 2022 la plataforma recibió alrededor de 1.2 millones de solicitudes de servicios de taxis de los cuales el 57% de esas solicitudes fueron abandonadas o rechazadas, lo que los llevó a analizar qué estaba pasando con la prestación de servicios internamente, en esa revisión realizada durante el mismo periodo de tiempo se obtuvieron las principales causas reportadas en la plataforma y que se listarán a continuación:

•	El usuario se fue

•	El vehículo se demoró en llegar

•	El usuario tomó otro taxi

Al identificar las causas de los servicios abandonados y rechazados, la empresa se planteó a resolver la siguiente pregunta al problema presentado.
¿Como reducir la alta cantidad de inconformidades de los usuarios recurrentes de la plataforma prediciendo sus comportamientos durante la semana y así programarles sus servicios de forma anticipada?

## Objetivo general
Diseñar un método para predecir las solicitudes de servicios de los usuarios recurrentes "de empresas de servicios de" taxis en la ciudad Bogotá - Colombia, "mediante" modelos de ML.


Evaluar el desempeño de modelos de ML para predecir
las solicitudes de servicios de los usuarios recurrentes de empresas de servicios de taxis en la ciudad Bogotá -  Colombia
basados en el comportamiento dentro de la plataforma.

## Objetivos específicos (Detallar las actividades)
• Revisar la literatura de al menos 10 artículos de proyectos relacionados (pendiente)

• Analizar los datos de los últimos 6 meses de servicios de la empresa taxis libres.

• Determinar las variables significativas a ser empleadas en los modelos de ML.

• Evaluar diferentes modelos de ML para la predicción de solicitudes.

## ¿Que solución propone al problema?
El diseño de un modelo predictivo de solicitudes de servicios de taxis basados en comportamientos de usuarios recurrentes de plataformas móviles para empresas de transporte

## ¿Cómo lo pretende solucionar?
Utilizando un modelo de clasificación donde se agrupe la información de los servicios que han tomado los usuarios por días de las semanas y horarios, determinando si en ese día y a esa hora se va a tomar un servicio.

## ¿Que resultados que espera obtener?
Se espera obtener listado por día de las semanas con todos los horarios del día y un determinador que indique si el servicio se va a tomar o no en ese horario. 

### Exploratory data analysis

Project library definition

In [1]:
!pip install holidays

  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/4d/bd/1869fa353dfaf3390f6d47df683d9a32a527803e5be54d74606be8b89eee/holidays-0.31-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.3/754.3 kB 44.9 MB/s eta 0:00:00


In [2]:
!pip install fastparquet

  Obtaining dependency information for fastparquet from https://files.pythonhosted.org/packages/f4/70/1756e78d37740ebb68c5705edf1339c2f674d2c9a36916f2caa107c07a8c/fastparquet-2023.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cramjam>=2.3 from https://files.pythonhosted.org/packages/30/79/abc388ed246721b7d56ef4410f07c8f81b1df5660f23f9b743cc6958ecdc/cramjam-2.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.9 MB/s eta 0:00:00


In [3]:
#librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.metrics import plot_confusion_matrix
from matplotlib.colors import ListedColormap
#from mlxtend.plotting import plot_decision_regions
#modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron

#from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold


import numpy as np
#from numpy import NaN
from datetime import datetime, timedelta
import holidays
#from os import name
import math
from multiprocessing import Pool
import time

The initial parameters are defined

In [4]:
TRAIN_WEEK = 6.0
TEST_WEEK = 1.0
BOGOTA_CODE = 11001
HOURS_OF_DAY = 24
WEEKS_RECURRENT = 5
FORMAT_DATE = '%Y-%m-%d'
FORMAT_COMPLETE_DATE = '%Y-%m-%d %H:%M:%S'
DATE_START_TRAINIG = pd.to_datetime("2023-03-10", format=FORMAT_DATE)
FICHERO_DATA = 's3://taxis-ml-2023-05-27/data/SERVICIO_UNIFICADO_2023.parquet.gzip'

#Se obtiene la fecha de inicial de entrenamiento
last_date_dataset = DATE_START_TRAINIG - timedelta(weeks=TRAIN_WEEK)
#Se obtiene la fecha de inicial de entrenamiento
first_date_dataset = DATE_START_TRAINIG + timedelta(weeks=TEST_WEEK)


The data is loaded into the dataFrame

In [5]:
#Se lee el archivo de un parquet a un dataframe
df_taxis = pd.read_parquet(FICHERO_DATA) 
#Se visualizan los datos
print(f"({len(df_taxis):,}) records were loaded into dataset!")

(16,511,429) records were loaded into dataset!


The data types of the dataset are displayed

In [6]:
df_taxis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16511429 entries, 0 to 16511428
Data columns (total 14 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID             int64  
 1   CITY           int64  
 2   USER           float64
 3   LATITUDEORI    float64
 4   LATITUDEDEST   float64
 5   LONGITUDEORI   float64
 6   LONGITUDEDEST  float64
 7   STATUS         object 
 8   CHANNEL        object 
 9   COMPLETEDATE   object 
 10  DATE           object 
 11  DAYOFWEEK      int64  
 12  HOUR           int64  
 13  MONTH          int64  
dtypes: float64(5), int64(5), object(4)
memory usage: 1.7+ GB


Validates how many null values are present in the dataset

In [7]:
df_taxis.isnull().sum()

ID                     0
CITY                   0
USER                 184
LATITUDEORI            0
LATITUDEDEST     3495636
LONGITUDEORI           0
LONGITUDEDEST    3495636
STATUS                 0
CHANNEL                0
COMPLETEDATE           0
DATE                   0
DAYOFWEEK              0
HOUR                   0
MONTH                  0
dtype: int64

Data is cleaned where the user is null

In [8]:
df_taxis = df_taxis[~df_taxis['USER'].isnull()]

Dataset fields are displayed

In [9]:
df_taxis.head(5)

,ID,CITY,USER,LATITUDEORI,LATITUDEDEST,LONGITUDEORI,LONGITUDEDEST,STATUS,CHANNEL,COMPLETEDATE,DATE,DAYOFWEEK,HOUR,MONTH
0,80206465,11001,8168434.0,4.646835,4.669337,-74.101619,-74.111349,CANCELADO,APP,2022-09-30 19:00:00,2022-09-30,5,19,9
1,80206466,11001,8692306.0,4.648861,NaN,-74.109828,NaN,ABANDONADO,APP,2022-09-30 19:00:00,2022-09-30,5,19,9
2,80206467,11001,10186684.0,4.607889,4.547080,-74.078844,-74.148150,ABANDONADO,APP,2022-09-30 19:00:00,2022-09-30,5,19,9
3,80206468,17001,10171299.0,5.052368,NaN,-75.480895,NaN,ABANDONADO,CHAT_BOT_659,2022-09-30 19:00:00,2022-09-30,5,19,9
4,80206469,17001,10244322.0,5.045227,NaN,-75.517250,NaN,ABANDONADO,CHAT_BOT_659,2022-09-30 19:00:00,2022-09-30,5,19,9


In [10]:
df_taxis.tail(5)

,ID,CITY,USER,LATITUDEORI,LATITUDEDEST,LONGITUDEORI,LONGITUDEDEST,STATUS,CHANNEL,COMPLETEDATE,DATE,DAYOFWEEK,HOUR,MONTH
16511424,96645776,11001,9519117.0,4.715707,4.670841,-74.128792,-74.068099,NUEVO,APP,2023-04-20 08:26:17,2023-04-20,4,8,4
16511425,96645777,11001,11359675.0,4.656801,NaN,-74.108836,NaN,NUEVO,RECEPCION,2023-04-20 08:26:18,2023-04-20,4,8,4
16511426,96645778,11001,10184549.0,4.706920,4.682351,-74.042674,-74.043735,NUEVO,APP,2023-04-20 08:26:19,2023-04-20,4,8,4
16511427,96645779,11001,11281923.0,4.618849,4.632341,-74.067500,-74.144769,NUEVO,APP,2023-04-20 08:26:19,2023-04-20,4,8,4
16511428,96645780,11001,9769386.0,4.721554,4.722161,-74.036146,-74.050731,NUEVO,APP,2023-04-20 08:26:19,2023-04-20,4,8,4


## Display of initial data

Dataset if filtred in final status and grouped calle,recepcion,web,externo, iontaxi, kisoko and reserva in others channels

In [11]:
df_taxis = df_taxis[(df_taxis['STATUS']=='ABANDONADO') | (df_taxis['STATUS']=='CUMPLIDO') | (df_taxis['STATUS']=='CANCELADO') | (df_taxis['STATUS']=='FINALIZADO')].replace({"CHAT_BOT_659": "CHATBOT", 
                  "CHAT_BOT_689": "CHATBOT",
                  "CHAT_BOT_729": "CHATBOT", 
                  "CHAT_BOT_803": "CHATBOT", 
                  "CALLE": "OTROS", 
                  "WEB": "OTROS", 
                  "EXTERNO": "OTROS", 
                  "IONTAXI": "OTROS", 
                  "Kiosko #1": "OTROS", 
                  "Reserva": "OTROS", 
                  "RECEPCION": "OTROS"})

In [12]:
tmp_taxis = df_taxis.copy()
tmp_taxis["DAYOFWEEK"] = df_taxis.DAYOFWEEK.replace({1: "MONDAY", 
                  2: "TUESDAY",
                  3: "WEDNESDAY", 
                  4: "THURSDAY", 
                  5: "FRIDAY", 
                  6: "SATURDAY", 
                  7: "SUNDAY"})
tmp_taxis["MONTH"] = df_taxis.MONTH.replace({1: "JANUARY", 
                  2: "FEBRUARY",
                  3: "MARCH", 
                  4: "APRIL", 
                  5: "MAY", 
                  6: "JUNE", 
                  7: "JULY", 
                  8: "AUGUST", 
                  9: "SEPTEMBER", 
                  10: "OCTOBER", 
                  11: "NOVEMBER",
                    12:"DECEMBER"})

The number of services in a period of time is displayed

## Display of filtered data

The dataset is filtered with the values to be used: 6 weeks for training and 1 week for the test

In [13]:
df_taxis = df_taxis[ (pd.to_datetime(df_taxis['COMPLETEDATE'], format=FORMAT_COMPLETE_DATE) >= last_date_dataset) & (pd.to_datetime(df_taxis['COMPLETEDATE'], format=FORMAT_COMPLETE_DATE) <= first_date_dataset) ]
df_taxis = df_taxis.sort_values(by=['COMPLETEDATE','USER'])
print(f"The dataset is restricted from date:{last_date_dataset} - to date:{first_date_dataset}")

The dataset is restricted from date:2023-01-27 00:00:00 - to date:2023-03-17 00:00:00


DataFrame data is displayed in a histogram to analyze the data

- Channels are limited to: APP, RECEPCION, CHAT_BOT_659, IVR and CALLE

- States are limited to: ABANDONADO, CUMPLIDO, CANCELADO and FINALIZADO

In [14]:
df_taxis = df_taxis[(df_taxis["CHANNEL"]=="APP") ]#| (df_taxis["CHANNEL"]=="RECEPCION") | (df_taxis["CHANNEL"]=="CHAT_BOT_659") | (df_taxis["CHANNEL"]=="IVR") | (df_taxis["CHANNEL"]=="CALLE")]
df_taxis = df_taxis[(df_taxis["STATUS"]=="ABANDONADO") | (df_taxis["STATUS"]=="CUMPLIDO") | (df_taxis["STATUS"]=="CANCELADO") | (df_taxis["STATUS"]=="FINALIZADO")]

In [15]:
df_taxis.drop(labels='STATUS', axis=1)

,ID,CITY,USER,LATITUDEORI,LATITUDEDEST,LONGITUDEORI,LONGITUDEDEST,CHANNEL,COMPLETEDATE,DATE,DAYOFWEEK,HOUR,MONTH
11416766,91623237,11001,9633983.0,4.677175,4.734961,-74.048048,-74.025712,APP,2023-01-27 00:00:04,2023-01-27,5,0,1
11416767,91623238,11001,8495182.0,4.691108,4.696928,-74.076157,-74.081983,APP,2023-01-27 00:00:07,2023-01-27,5,0,1
11416769,91623240,11001,8121136.0,4.679135,4.708913,-74.053129,-74.137175,APP,2023-01-27 00:00:19,2023-01-27,5,0,1
11416768,91623239,11001,11126612.0,4.637783,4.630673,-74.156330,-74.159206,APP,2023-01-27 00:00:19,2023-01-27,5,0,1
11416770,91623241,11001,8218555.0,4.584267,4.500195,-74.091763,-74.109510,APP,2023-01-27 00:00:29,2023-01-27,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867706,95002058,11001,11105533.0,4.709767,4.675948,-74.112223,-74.092874,APP,2023-03-16 23:59:11,2023-03-16,4,23,3
14867707,95002059,11001,8402040.0,4.638548,4.641556,-74.161491,-74.165568,APP,2023-03-16 23:59:20,2023-03-16,4,23,3
14867709,95002061,11001,11282492.0,4.569968,4.574290,-74.145076,-74.124000,APP,2023-03-16 23:59:34,2023-03-16,4,23,3
14867710,95002062,11001,8051106.0,4.646117,4.648542,-74.086899,-74.085399,APP,2023-03-16 23:59:52,2023-03-16,4,23,3


In [16]:
df_taxis = df_taxis.drop(columns=["CITY","ID","LATITUDEDEST","LONGITUDEDEST","LATITUDEORI","LONGITUDEORI","COMPLETEDATE","CHANNEL","STATUS","MONTH"])
#Duplicate values are removed from the dataframe and sorted by the most relevant fields
df_taxis = df_taxis.drop_duplicates().sort_values(by=['DATE','USER','DAYOFWEEK','HOUR'])
#The data frame is reindexed again
df_taxis = df_taxis.reset_index(drop=True)
#The data frame is displayed
df_taxis.head(5)

,USER,DATE,DAYOFWEEK,HOUR
0,3564.0,2023-01-27,5,18
1,3598.0,2023-01-27,5,6
2,5174.0,2023-01-27,5,16
3,5174.0,2023-01-27,5,22
4,7004.0,2023-01-27,5,18


Non-recurrent users are removed from the dataset (Recurrent users are those who took at least 6 services on the same day and at the same time during the last 8 previous weeks)

In [17]:
def recurrentUsers(p_user, df_tx, training_date):
    current_week = training_date.isoweekday()
    first_time = pd.to_datetime(training_date, format=FORMAT_DATE)
    last_time = first_time + timedelta(weeks=-TRAIN_WEEK)
    for hour in range(HOURS_OF_DAY):
      df_rec = df_tx[ (df_tx["USER"] == p_user) & (df_tx["DAYOFWEEK"] == current_week) & (df_tx['DATE'].astype(str) >= last_time.strftime(FORMAT_DATE)) & (df_tx['DATE'].astype(str) <= first_time.strftime(FORMAT_DATE)) & (df_tx['HOUR'] == hour) ]
      total = len(df_rec)
      if total >= WEEKS_RECURRENT:
        return df_tx[ (df_tx["USER"] == p_user) ]
    return pd.DataFrame({})

In [18]:
def processingRecurrentUsers(data_taxis):
    start_time = time.perf_counter()
    df_user = data_taxis.loc[:,['USER']].copy()
    df_new = pd.DataFrame({})
    total_user, removed_users, count, user_array = 0, 0, 0, []
    list_users = df_user.value_counts().index.to_list()
    print(f"Users to process {len(list_users)}")

    for user in list_users:
        count += 1
        user_array.append(user[0])
        if count % 10000 == 0 or count >= len(list_users):
            list_result = []
            args = [(user, data_taxis[ (data_taxis["USER"] == user) ], DATE_START_TRAINIG) for user in user_array]
            with Pool() as pool:
                list_result = pool.starmap(recurrentUsers, args)
            for df in list_result:
                if len(df) > 0:
                    total_user += 1
                else:
                    removed_users += 1
                df_new = pd.concat([df_new, df])
            user_array = []
            finish_time = time.perf_counter()
            print(f"Processing first {count} users of {len(df_user.value_counts().index.to_list())-count}, total recurents users: {total_user}, removed users {removed_users} in {format(finish_time-start_time)} seconds")
        if count >= 10000:
            break
    finish_time = time.perf_counter()
    print(f"Program finished in {format(finish_time-start_time)} seconds, total recurents users: {total_user}, removed users {removed_users} total rows {len(df_new)}")
    return df_new

Processing recurrent users

In [19]:
df_taxis = processingRecurrentUsers(df_taxis)

Users to process 214190
Processing first 10000 users of 204190, total recurents users: 894, removed users 9106 in 116.09888397300006 seconds
Program finished in 116.29362704100004 seconds, total recurents users: 894, removed users 9106 total rows 44504


### DATA VISUALIZATION

* Scatter plot of the variables
* Graph of distribution of variables (Low distribution, Low prediction)
* Correlation graphs (Heat map)

The relationship between variables in the data are displayed

Function to train the model and predict with test values

In [20]:
def predictModel(model, X_train, y_train, X_test):
    #A continuación se entrena el modelo
    model.fit(X_train, y_train)
    #Se crean las predicciones para pruebas
    y_pred = model.predict(X_test)
    #Se crean las probabilidades para pruebas
    y_prob = model.predict_proba(X_test)
    return model, y_pred, y_prob

Function that evaluate the K-neighbors model

In [21]:
def knnModel(X_train, y_train, X_test):
    #Se aplica el modelo de K vecinos
    model = KNeighborsClassifier(n_neighbors=2)#, metric='euclidean',leaf_size=1, p=1, weights='uniform')
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Multi-layer Perceptron model

In [22]:
def mlpModel(X_train, y_train, X_test):
    #Se aplica el modelo de perceptrones multi capa
    model = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(30,30, 30), random_state=1,learning_rate_init=0.001,max_iter=5000)
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Perceptron model

In [23]:
def perModel(X_train, y_train, X_test):
    #Se aplica el modelo de perceptrones multi capa
    model = Perceptron(eta0=0.1, n_iter_no_change=10, random_state=1)
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Logistic Regression model

In [24]:
def lrModel(X_train, y_train, X_test):
    #Se aplica el modelo de de regesión logística
    model = LogisticRegression(random_state = 1)
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Random Forest model

In [25]:
def rfcModel(X_train, y_train, X_test):
    #Se aplica el modelo de los arboles aleatorios
    model = RandomForestClassifier(criterion='gini', max_depth=5, n_estimators=200, class_weight="balanced")
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Support Vector Machine model

In [26]:
def svcModel(X_train, y_train, X_test):
    #Se aplica el modelo de máquina de soporte de vectores
    model = SVC(kernel='rbf', random_state=1, gamma=0.2, C=1.0, class_weight='balanced')#, class_weight='balanced'
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Decision Tree model

In [27]:
def dtcModel(X_train, y_train, X_test):
    #Se aplica el modelo de los arboles de desiciones
    model = DecisionTreeClassifier(criterion='gini', max_depth=3, class_weight='balanced')#class_weight={0: 1, 1: 5}
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate Extra Trees Classifier

In [28]:
def etcModel(X_train, y_train, X_test):
    model = ExtraTreesClassifier(n_estimators=1, max_depth=1, min_samples_split=2, random_state=0, class_weight="balanced")
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate Gradient Boosting Classifier, this function supports both binary and multi-class classification

In [29]:
def gbcModel(X_train, y_train, X_test):
    model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate Ada Boost

In [30]:

def abcModel(X_train, y_train, X_test):
    model = AdaBoostClassifier(n_estimators=1,algorithm='SAMME',random_state=0)
    return predictModel(model, X_train, y_train, X_test)

Function that evaluate the Stochastic Gradient Descent model

In [31]:
def sgdModel(X_train, y_train, X_test,):
    #Se aplica el modelo de gradiente descendente estocástico
    model =SGDClassifier(loss='huber', random_state=1, max_iter=2000, epsilon=0.1)#loss='hinge'
    return predictModel(model, X_train, y_train, X_test)

Function to display the confusion matrix

In [32]:
def confusionMatrix(model, y_test, y_pred):
  plt.rcParams.update({'font.size': 16})
  cm = confusion_matrix(y_test, y_pred, labels=model.classes_)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                display_labels=model.classes_)
  disp.plot()
  plt.show()

Function to calculate a user's probability

In [33]:
def ProbabilisticClassifier(holiday, hour, dayOfWeek, data):
  df_onset = data[ (data['HOLIDAY'] == holiday) & (data['DAYOFWEEK'] == dayOfWeek) & (data['HOUR'] == hour) & (data['SERVICE'] == 1) ]
  total_onset = len(df_onset)
  if total_onset > 0:
    return total_onset / TRAIN_WEEK
  else:
    return 0

Function to evaluate the probabilistic model

In [34]:
def probabilisticModel(data, date_predict):
    co_holidays = holidays.CO()
    threshold = 0.7
    array_result = []
    array_prob = []
    for hour in range(HOURS_OF_DAY):
        result = ProbabilisticClassifier(1 if co_holidays.get(date_predict) != None else 0, hour, date_predict.isoweekday(), data)
        array_prob.append(round(result,2))
        array_result.append(1 if result >= threshold else 0)
    return array_result, array_prob

Function to validate if it contains all data in 0

In [35]:
def existAllZero(arr):
    if len(arr)>0:
        for value in arr:
            if value != 0:
                return False
    return True

Function to return the name of the models

In [36]:
def modelName(model_type):
    if model_type == 'LR':
        return 'Logistic regression'
    elif model_type == 'KNN':
        return 'Kneighbors'
    elif model_type == 'MLP':
        return 'Multilayer perceptron'
    elif model_type == 'RFC':
        return 'Random forest'
    elif model_type == 'SVC':
        return 'Support vector'
    elif model_type == 'SGD':
        return 'Gradient descent stochastic'
    elif model_type == 'DTC':
        return 'Decision tree'
    elif model_type == 'GBC':
        return 'Gradient boosting'
    elif model_type == 'ETC':
        return 'Extra trees'
    elif model_type == 'ABC':
        return 'Ada Boost'
    elif model_type == 'PR':
        return 'Probabilistic'
    else:
        return 'NA'

Function to fill the dataset with the services taken in 1 and those not taken in 0

In [37]:
def setDataService(data, from_date, to_date):
  co_holidays = holidays.CO()
  remaining_days = to_date.date() - from_date.date()
  service_array = []
  existService = 0
  for i in range(remaining_days.days+1):
    last_time = from_date + timedelta(days=i)
    if last_time.date().isoweekday() == to_date.date().isoweekday():
      for hour in range(HOURS_OF_DAY):
        #Se filtra por el mismo día de la semana, fecha y hora
        #df_service = data[(data['DIADESEMANA'] == last_time.isoweekday()) & (data['FECHA'] == last_time.date()) & (data['HORA'] == hour) ].to_numpy()
        df_service = data[ (data['DATE'].astype(str) == last_time.strftime(FORMAT_DATE)) & (data['HOUR'] == hour) ]
        if len(df_service) > 0:
          #print(f"fecha con servicio ={last_time.strftime(FORMAT_DATE)}, hour ={hour}")
          #Se llena con valor en 1 porqué se encontró un servicio
          existService = 1
        else:
          #print(f"fecha sin servicio ={last_time.strftime(FORMAT_DATE)}, hour ={hour}")
          #Se llena con valor en 0 porqué no se encontró ningún servicio
          existService = 0        
        df_service = [1 if co_holidays.get(last_time) != None else 0,last_time.date().strftime(FORMAT_DATE),last_time.isoweekday(),hour,existService]
        service_array.append(df_service)
  return pd.DataFrame(service_array, 
             columns=['HOLIDAY','DATE','DAYOFWEEK','HOUR','SERVICE'])

# Hyperparameters

KNN

In [45]:
start_proccess = datetime.now()
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,7))
n_neighbors = list(range(1,10))
p=[1,2]
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

list_users = df_taxis.loc[:,['USER']].head(10).value_counts().index.to_list()

print(f"Inició el proceso: {datetime.now()}")
start_date = datetime.now()
confusion_phone = np.matrix(0,dtype=int)
y_test_arr = np.array(0,dtype=int)
y_prob_arr = np.array(0,dtype=float)
for user in list_users:
    print(f'---------------------------{user}------------------------------')
    remaining_days = DATE_START_TRAINIG.date() + timedelta(days=1) - DATE_START_TRAINIG.date()
    for i in range(remaining_days.days):
        last_time = DATE_START_TRAINIG.date() + timedelta(days=i)

        #Se obtiene la fecha de inicial de entrenamiento
        date_first_training = pd.to_datetime(last_time, format=FORMAT_DATE) - timedelta(weeks=TRAIN_WEEK)

        data = df_taxis[df_taxis["USER"]==user].copy()

        #Se llenan en 0 las horas de los servicios que no se tomaron por el usuario
        data = setDataService(data, date_first_training, pd.to_datetime(last_time, format=FORMAT_DATE))
        #Se crear el dataframe de entrenamiento desde la fecha inicial hasta la fecha de predicción
        train = data[ (pd.to_datetime(data['DATE'], format=FORMAT_DATE) >= date_first_training) & (pd.to_datetime(data['DATE'], format=FORMAT_DATE) < pd.to_datetime(last_time, format=FORMAT_DATE)) ]
        #Se crear el dataframe de pruebas desde la fecha de predicción hasta una semana adelante
        test = data[(pd.to_datetime(data['DATE'], format=FORMAT_DATE) == pd.to_datetime(last_time, format=FORMAT_DATE)) ]

        #Data for testing the models
        train = train.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]
        test = test.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]

        y_train = train.SERVICE
        X_train = train.drop(labels='SERVICE', axis=1)

        y_test = test.SERVICE
        X_test = test.drop(labels='SERVICE', axis=1)

        try:
            #Convert to dictionary
            hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p, weights=weights, metric=metric)
            #Create new KNN object
            model = KNeighborsClassifier()
            #Use GridSearch
            clf = GridSearchCV(model, hyperparameters, cv=10, scoring='accuracy',error_score=0)
            #Fit the model
            best_model = clf.fit(X_train,y_train)
            #Print The value of best Hyperparameters
            print(f'---------------------------{last_time}------------------------------')
            print("Best: %f using %s" % (best_model.best_score_, best_model.best_params_))
            print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
            print('Best p:', best_model.best_estimator_.get_params()['p'])
            print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
            print('Best weights:', best_model.best_estimator_.get_params()['weights'])
            print('Best metric:', best_model.best_estimator_.get_params()['metric'])
        except Exception as err:
            print(f"Error predict  parameter: {err}")

Inició el proceso: 2023-09-02 14:54:54.558554
---------------------------(9361311.0,)------------------------------
---------------------------2023-03-10------------------------------
Best: 0.859524 using {'leaf_size': 1, 'metric': 'euclidean', 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
Best leaf_size: 1
Best p: 1
Best n_neighbors: 5
Best weights: uniform
Best metric: euclidean


GBC

In [46]:
start_proccess = datetime.now()
#List Hyperparameters that we want to tune.
max_depth = list(range(1,7))
n_estimators = [10,20,30,50,80,100,150]
learning_rate = [0.1,0.3,0.5,0.8,1.0]
random_state = [0,1]
max_features=['sqrt','log2']

list_users = df_taxis.loc[:,['USER']].head(10).value_counts().index.to_list()

print(f"Inició el proceso: {datetime.now()}")
start_date = datetime.now()
confusion_phone = np.matrix(0,dtype=int)
y_test_arr = np.array(0,dtype=int)
y_prob_arr = np.array(0,dtype=float)
for user in list_users:
    print(f'---------------------------{user}------------------------------')
    remaining_days = DATE_START_TRAINIG.date() + timedelta(days=1) - DATE_START_TRAINIG.date()
    for i in range(remaining_days.days):
        last_time = DATE_START_TRAINIG.date() + timedelta(days=i)

        #Se obtiene la fecha de inicial de entrenamiento
        date_first_training = pd.to_datetime(last_time, format=FORMAT_DATE) - timedelta(weeks=TRAIN_WEEK)

        data = df_taxis[df_taxis["USER"]==user].copy()

        #Se llenan en 0 las horas de los servicios que no se tomaron por el usuario
        data = setDataService(data, date_first_training, pd.to_datetime(last_time, format=FORMAT_DATE))
        #Se crear el dataframe de entrenamiento desde la fecha inicial hasta la fecha de predicción
        train = data[ (pd.to_datetime(data['DATE'], format=FORMAT_DATE) >= date_first_training) & (pd.to_datetime(data['DATE'], format=FORMAT_DATE) < pd.to_datetime(last_time, format=FORMAT_DATE)) ]
        #Se crear el dataframe de pruebas desde la fecha de predicción hasta una semana adelante
        test = data[(pd.to_datetime(data['DATE'], format=FORMAT_DATE) == pd.to_datetime(last_time, format=FORMAT_DATE)) ]

        #Data for testing the models
        train = train.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]
        test = test.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]

        y_train = train.SERVICE
        X_train = train.drop(labels='SERVICE', axis=1)

        y_test = test.SERVICE
        X_test = test.drop(labels='SERVICE', axis=1)

        try:
            #Convert to dictionary
            hyperparameters = dict(n_estimators=n_estimators, learning_rate=learning_rate, random_state=random_state, max_depth=max_depth, max_features=max_features)
            #Create new GBC object
            model = GradientBoostingClassifier()
            #Use GridSearch
            clf = GridSearchCV(model, hyperparameters, cv=10, scoring='accuracy',error_score=0)
            #Fit the model
            best_model = clf.fit(X_train,y_train)
            #Print The value of best Hyperparameters
            print(f'---------------------------{last_time}------------------------------')
            print("Best: %f using %s" % (best_model.best_score_, best_model.best_params_))
            print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
            print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
            print('Best random_state:', best_model.best_estimator_.get_params()['random_state'])
            print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
            print('Best max_features:', best_model.best_estimator_.get_params()['max_features'])
        except Exception as err:
            print(f"Error predict  parameter: {err}")

Inició el proceso: 2023-09-02 14:55:25.454164
---------------------------(9361311.0,)------------------------------
---------------------------2023-03-10------------------------------
Best: 0.859524 using {'learning_rate': 0.1, 'max_depth': 2, 'max_features': 'sqrt', 'n_estimators': 80, 'random_state': 0}
Best n_estimators: 80
Best learning_rate: 0.1
Best random_state: 0
Best max_depth: 2
Best max_features: sqrt


MLP

In [47]:
start_proccess = datetime.now()
#List Hyperparameters that we want to tune.
random_state = [0,1]
solver=['lbfgs', 'sgd', 'adam']
max_iter= [100]#,500,800,1000,3000,5000,8000
learning_rate_init=[0.001]
hidden_layer_sizes=[(30,30,30)]#,(10,20,30),(20,20,20),(10,10,10),(40,40,40)
alpha=[1e-2]#,1e-3,1e-4,1e-5
activation=['identity', 'logistic', 'tanh', 'relu']
learning_rate=['constant', 'invscaling', 'adaptive']
batch_size=[100]#,300,400,500

list_users = df_taxis.loc[:,['USER']].head(1).value_counts().index.to_list()

print(f"Inició el proceso: {datetime.now()}")
start_date = datetime.now()
confusion_phone = np.matrix(0,dtype=int)
y_test_arr = np.array(0,dtype=int)
y_prob_arr = np.array(0,dtype=float)
for user in list_users:
    print(f'---------------------------{user}------------------------------')
    remaining_days = DATE_START_TRAINIG.date() + timedelta(days=1) - DATE_START_TRAINIG.date()
    for i in range(remaining_days.days):
        last_time = DATE_START_TRAINIG.date() + timedelta(days=i)

        #Se obtiene la fecha de inicial de entrenamiento
        date_first_training = pd.to_datetime(last_time, format=FORMAT_DATE) - timedelta(weeks=TRAIN_WEEK)

        data = df_taxis[df_taxis["USER"]==user].copy()

        #Se llenan en 0 las horas de los servicios que no se tomaron por el usuario
        data = setDataService(data, date_first_training, pd.to_datetime(last_time, format=FORMAT_DATE))
        #Se crear el dataframe de entrenamiento desde la fecha inicial hasta la fecha de predicción
        train = data[ (pd.to_datetime(data['DATE'], format=FORMAT_DATE) >= date_first_training) & (pd.to_datetime(data['DATE'], format=FORMAT_DATE) < pd.to_datetime(last_time, format=FORMAT_DATE)) ]
        #Se crear el dataframe de pruebas desde la fecha de predicción hasta una semana adelante
        test = data[(pd.to_datetime(data['DATE'], format=FORMAT_DATE) == pd.to_datetime(last_time, format=FORMAT_DATE)) ]

        #Data for testing the models
        train = train.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]
        test = test.loc[:,['HOLIDAY','DAYOFWEEK','HOUR','SERVICE']]

        y_train = train.SERVICE
        X_train = train.drop(labels='SERVICE', axis=1)

        y_test = test.SERVICE
        X_test = test.drop(labels='SERVICE', axis=1)

        try:
            #Convert to dictionary
            hyperparameters = dict(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state, learning_rate_init=learning_rate_init, max_iter=max_iter, activation=activation, learning_rate=learning_rate, batch_size=batch_size)
            #Create new MLP object
            model = MLPClassifier()
            #Use GridSearch
            clf = GridSearchCV(model, hyperparameters, cv=10, scoring='accuracy',error_score=0)
            #Fit the model
            best_model = clf.fit(X_train,y_train)
            #Print The value of best Hyperparameters
            print(f'---------------------------{last_time}------------------------------')
            print("Best: %f using %s" % (best_model.best_score_, best_model.best_params_))
            print('Best solver:', best_model.best_estimator_.get_params()['solver'])
            print('Best alpha:', best_model.best_estimator_.get_params()['alpha'])
            print('Best hidden_layer_sizes:', best_model.best_estimator_.get_params()['hidden_layer_sizes'])
            print('Best random_state:', best_model.best_estimator_.get_params()['random_state'])
            print('Best learning_rate_init:', best_model.best_estimator_.get_params()['learning_rate_init'])
            print('Best max_iter:', best_model.best_estimator_.get_params()['max_iter'])
            print('Best activation:', best_model.best_estimator_.get_params()['activation'])
            print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
            print('Best batch_size:', best_model.best_estimator_.get_params()['batch_size'])
            
        except Exception as err:
            print(f"Error predict  parameter: {err}")

Inició el proceso: 2023-09-02 15:02:31.655941
---------------------------(9361311.0,)------------------------------


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: Converg

---------------------------2023-03-10------------------------------
Best: 0.784286 using {'activation': 'relu', 'alpha': 0.01, 'batch_size': 100, 'hidden_layer_sizes': (30, 30, 30), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 100, 'random_state': 1, 'solver': 'lbfgs'}
Best solver: lbfgs
Best alpha: 0.01
Best hidden_layer_sizes: (30, 30, 30)
Best random_state: 1
Best learning_rate_init: 0.001
Best max_iter: 100
Best activation: relu
Best learning_rate: constant
Best batch_size: 100


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
